In [22]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

import os
import torch

from utils import load_segment, visualize_cropper, compute_PCA, visualize_PCA

# Load the model
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14_reg').cuda()
model.eval()

%matplotlib widget

Using cache found in /home/jgcarrasco/.cache/torch/hub/facebookresearch_dinov2_main


# Config

Below you can change several parameters. I recommend you to play with the number of PCA components and the crop size.

In [16]:
PCA_DEPTH = False # Whether to compute PCA across al layers, or layerwise. I obtained better results with layerwise (i.e. False)
N_COMPONENTS = 5 # Number of PCA components 
CROP_SIZE = 600 # Size of the crop that we want to compute PCA on. The crop size will be rounded to the smallest multiple of the patch size
PATCH_SIZE = 14 # patch size that the DINO model uses, do not change!
CROP_SIZE = (CROP_SIZE // PATCH_SIZE) * PATCH_SIZE

Now we are going to read the data, which has to be stored in the following format:

```
segments/
    segment_id/
        layers/
            00.tif
            ...
            64.tif
        segment_id_inklabels.png
        segment_id_mask.png
```

**NOTE:** This implementation is a really inefficient prototype! Try to load smaller segments first, below I provide different suggestions. For now, we will just work with the segment containing Casey's pi:

In [26]:
#segment_id = "20230509182749" # casey pi's segment
segment_id = "20230519195952" # another small segment
#segment_id = "20231210121321" # small segment with lots of predicted letters. This starts to get larger and might not work in colab

# load the segment
tif, inklabels = load_segment(segment_id)

Execute the cell below to visualize the segment. You can click on the segment to move the red square. Once that you have selected a region of your interest, press enter to crop it and keep following the notebook. Try to crop the whole region that was predicted as a pi!

In [27]:
result = visualize_cropper(tif, inklabels, crop_size=CROP_SIZE)

Subimage saved!


Now we can compute the PCA vectors on the cropped region:

In [28]:
cropped_tif = result["cropped_tif"]
cropped_inklabels = result["cropped_inklabels"]
y_pca = compute_PCA(cropped_tif, model, PCA_DEPTH, n_components=N_COMPONENTS)

Finally, we visualize the results. Try to localize hints of the letter. If you cannot see anything, use the slider to move through the different layers, ensure that you have cropped the whole pi or try cropping a larger area. Now you can try visualizing other small segments/regions and try to discover letters!

In [29]:
visualize_PCA(y_pca, cropped_tif, cropped_inklabels)

interactive(children=(IntSlider(value=33, description='Slice Index', max=64), Output()), _dom_classes=('widget…